# Project : Optimization for Machine Learning

### Initial Setup

In [91]:
!pip install torch -q 
!pip install torchvision -q
!pip install optuna -q
!pip install matplotlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 23.4 MB/s eta 0:00:00:00:0100:01


In [85]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import random
import optuna
import optuna.visualization as vis
import time
import itertools
import os

In [3]:
# Code reproducibility
def set_seed(seed=1):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    set_seed(worker_seed)

In [4]:
BATCH_SIZE = 256
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Dataset Preprocessing and CNN benchmark model

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

gen = torch.Generator()
gen.manual_seed(1)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# For reproductibility the train_loader will be initialized in the main function

100%|██████████| 170M/170M [00:03<00:00, 48.7MB/s] 


In [7]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

### Training and testing function

In [1]:
def train(model, optimizer, criterion, train_loader, epochs, trial=None):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
        ### For the optuna study we make a report at each epoch and we prune trials with non-optimal hyperparameters
        if trial is not None:
            trial.report(avg_loss, step=epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()
    return avg_loss

In [9]:
def test(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return correct / total

### Comparison of optimizers

In [10]:
optimizers_list = {

    'SGD': lambda model, trial: optim.SGD(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
    ),

    'AdaGrad': lambda model, trial: optim.Adagrad(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
    ),

    'RMSprop': lambda model, trial: optim.RMSprop(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        alpha=trial.suggest_float("alpha", 0.8, 0.99),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    ),

    'Adam': lambda model, trial: optim.Adam(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        betas=(
            trial.suggest_float("beta1", 0.85, 0.99),
            trial.suggest_float("beta2", 0.9, 0.999)
        ),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    ),

    'AMSGrad': lambda model, trial: optim.Adam(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        betas=(
            trial.suggest_float("beta1", 0.85, 0.99),
            trial.suggest_float("beta2", 0.9, 0.999)
        ),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
        amsgrad=True
    ),

    'AdamW': lambda model, trial: optim.AdamW(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        betas=(
            trial.suggest_float("beta1", 0.85, 0.99),
            trial.suggest_float("beta2", 0.9, 0.999)
        ),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    ),

    'Nadam': lambda model, trial: optim.NAdam(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        betas=(
            trial.suggest_float("beta1", 0.85, 0.99),
            trial.suggest_float("beta2", 0.9, 0.999)
        ),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True),
    ),

    'RAdam': lambda model, trial: optim.RAdam(
        model.parameters(),
        lr=trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        betas=(
            trial.suggest_float("beta1", 0.85, 0.99),
            trial.suggest_float("beta2", 0.9, 0.999)
        ),
        weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    ),
}

In [ ]:
def run_optuna_study(optimizer_name, n_trials):

    # The objective function that we seek to optimize in the optuna study is the accuracy
    def objective(trial):
        set_seed(1)
        model = CNN_model().to(DEVICE)
        optimizer = optimizers_list[optimizer_name](model, trial)
        criterion = torch.nn.CrossEntropyLoss()
        # For reproductibility we set the training
        gen = torch.Generator()
        gen.manual_seed(1)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, generator=gen, worker_init_fn=seed_worker)
        train(model, optimizer, criterion, train_loader, epochs=EPOCHS, trial=trial)
        acc = test(model)
        return acc
        
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=1))
    start_time = time.time()
    
    study.optimize(objective, n_trials=n_trials)
    elapsed_time = time.time() - start_time

    print(f"\n Results for {optimizer_name} ")
    print("Best hyperparameters :", study.best_params)
    print(f"Best accuracy : {study.best_value:.2%}")
    print(f"Execution time : {elapsed_time:.2f} secondes")
    return study

In [12]:
SGD_study=run_optuna_study("SGD", 20)

[I 2025-06-13 01:16:28,733] A new study created in memory with name: no-name-407229f1-ffa9-4b76-b5f9-6f0b3e213509


Epoch 1/10 - Loss: 2.3019
Epoch 2/10 - Loss: 2.2967
Epoch 3/10 - Loss: 2.2917
Epoch 4/10 - Loss: 2.2865
Epoch 5/10 - Loss: 2.2811
Epoch 6/10 - Loss: 2.2754
Epoch 7/10 - Loss: 2.2694
Epoch 8/10 - Loss: 2.2628
Epoch 9/10 - Loss: 2.2557
Epoch 10/10 - Loss: 2.2477


[I 2025-06-13 01:19:01,333] Trial 0 finished with value: 0.2233 and parameters: {'lr': 0.00046568046379195655, 'weight_decay': 0.0014486833887239336}. Best is trial 0 with value: 0.2233.


Epoch 1/10 - Loss: 2.3045
Epoch 2/10 - Loss: 2.3044
Epoch 3/10 - Loss: 2.3043
Epoch 4/10 - Loss: 2.3042
Epoch 5/10 - Loss: 2.3040
Epoch 6/10 - Loss: 2.3039
Epoch 7/10 - Loss: 2.3038
Epoch 8/10 - Loss: 2.3037
Epoch 9/10 - Loss: 2.3036
Epoch 10/10 - Loss: 2.3035


[I 2025-06-13 01:21:33,526] Trial 1 finished with value: 0.1027 and parameters: {'lr': 1.0010539860510485e-05, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.2233.


...

Epoch 1/10 - Loss: 2.3037


[I 2025-06-13 01:37:04,642] Trial 18 pruned. 


Epoch 1/10 - Loss: 2.2453


[I 2025-06-13 01:37:19,444] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.8518

 Results for SGD 
Best hyperparameters : {'lr': 0.00046568046379195655, 'weight_decay': 0.0014486833887239336}
Best accuracy : 22.33%
Execution time : 1250.71 secondes


In [13]:
AdaGrad_study=run_optuna_study("AdaGrad", 20)

[I 2025-06-13 01:37:19,451] A new study created in memory with name: no-name-519eacc6-5536-4b3b-94b4-3f4381407986


Epoch 1/10 - Loss: 1.7700
Epoch 2/10 - Loss: 1.5703
Epoch 3/10 - Loss: 1.4967
Epoch 4/10 - Loss: 1.4487
Epoch 5/10 - Loss: 1.4172
Epoch 6/10 - Loss: 1.3913
Epoch 7/10 - Loss: 1.3707
Epoch 8/10 - Loss: 1.3519
Epoch 9/10 - Loss: 1.3369
Epoch 10/10 - Loss: 1.3228


[I 2025-06-13 01:39:50,479] Trial 0 finished with value: 0.5263 and parameters: {'lr': 0.00046568046379195655, 'weight_decay': 0.0014486833887239336}. Best is trial 0 with value: 0.5263.


Epoch 1/10 - Loss: 2.2811
Epoch 2/10 - Loss: 2.2570
Epoch 3/10 - Loss: 2.2390
Epoch 4/10 - Loss: 2.2233
Epoch 5/10 - Loss: 2.2092
Epoch 6/10 - Loss: 2.1960
Epoch 7/10 - Loss: 2.1837
Epoch 8/10 - Loss: 2.1722
Epoch 9/10 - Loss: 2.1615
Epoch 10/10 - Loss: 2.1512


[I 2025-06-13 01:42:20,872] Trial 1 finished with value: 0.3051 and parameters: {'lr': 1.0010539860510485e-05, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.5263.


...

Epoch 1/10 - Loss: 1.5580


[I 2025-06-13 02:06:53,381] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.7939


[I 2025-06-13 02:07:08,195] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.6232

 Results for AdaGrad 
Best hyperparameters : {'lr': 0.02284910284964674, 'weight_decay': 0.0021752997681340367}
Best accuracy : 60.12%
Execution time : 1788.74 secondes


In [14]:
RMSprop_study=run_optuna_study("RMSprop", 20)

[I 2025-06-13 02:07:08,201] A new study created in memory with name: no-name-d62a04da-ef24-4818-a69e-f81a803e2b68


Epoch 1/10 - Loss: 1.5665
Epoch 2/10 - Loss: 1.2106
Epoch 3/10 - Loss: 1.0464
Epoch 4/10 - Loss: 0.9253
Epoch 5/10 - Loss: 0.8326
Epoch 6/10 - Loss: 0.7492
Epoch 7/10 - Loss: 0.6786
Epoch 8/10 - Loss: 0.6048
Epoch 9/10 - Loss: 0.5375
Epoch 10/10 - Loss: 0.4765


[I 2025-06-13 02:09:38,983] Trial 0 finished with value: 0.7225 and parameters: {'lr': 0.00046568046379195655, 'alpha': 0.93686165375401, 'weight_decay': 1.0007903854383775e-05}. Best is trial 0 with value: 0.7225.


Epoch 1/10 - Loss: 1.7251
Epoch 2/10 - Loss: 1.4396
Epoch 3/10 - Loss: 1.3174
Epoch 4/10 - Loss: 1.2273
Epoch 5/10 - Loss: 1.1576
Epoch 6/10 - Loss: 1.1000
Epoch 7/10 - Loss: 1.0519
Epoch 8/10 - Loss: 1.0059
Epoch 9/10 - Loss: 0.9676
Epoch 10/10 - Loss: 0.9315


[I 2025-06-13 02:12:07,898] Trial 1 finished with value: 0.6318 and parameters: {'lr': 0.0001619311091244073, 'alpha': 0.8278836192552516, 'weight_decay': 1.8924123965722247e-05}. Best is trial 0 with value: 0.7225.


...

Epoch 1/10 - Loss: 1.6531


[I 2025-06-13 02:38:55,066] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.7671
Epoch 1/10 - Loss: 2.0666
Epoch 2/10 - Loss: 1.5696
Epoch 3/10 - Loss: 1.4391
Epoch 4/10 - Loss: 1.3664
Epoch 5/10 - Loss: 1.3365
Epoch 6/10 - Loss: 1.3026
Epoch 7/10 - Loss: 1.2799
Epoch 8/10 - Loss: 1.2595
Epoch 9/10 - Loss: 1.2486
Epoch 10/10 - Loss: 1.2198


[I 2025-06-13 02:41:26,363] Trial 19 finished with value: 0.467 and parameters: {'lr': 0.0033533627242136807, 'alpha': 0.948356002096577, 'weight_decay': 0.004810170043087906}. Best is trial 0 with value: 0.7225.



 Results for RMSprop 
Best hyperparameters : {'lr': 0.00046568046379195655, 'alpha': 0.93686165375401, 'weight_decay': 1.0007903854383775e-05}
Best accuracy : 72.25%
Execution time : 2058.16 secondes


In [15]:
Adam_study=run_optuna_study("Adam", 20)

[I 2025-06-13 02:41:26,369] A new study created in memory with name: no-name-c56bf2b0-94a9-4815-a638-88ee66d9f588


Epoch 1/10 - Loss: 1.5251
Epoch 2/10 - Loss: 1.1734
Epoch 3/10 - Loss: 1.0173
Epoch 4/10 - Loss: 0.9055
Epoch 5/10 - Loss: 0.8234
Epoch 6/10 - Loss: 0.7478
Epoch 7/10 - Loss: 0.6769
Epoch 8/10 - Loss: 0.6056
Epoch 9/10 - Loss: 0.5424
Epoch 10/10 - Loss: 0.4802


[I 2025-06-13 02:43:56,434] Trial 0 finished with value: 0.7264 and parameters: {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.7264.


Epoch 1/10 - Loss: 1.9210
Epoch 2/10 - Loss: 1.6318
Epoch 3/10 - Loss: 1.5178
Epoch 4/10 - Loss: 1.4392
Epoch 5/10 - Loss: 1.3856
Epoch 6/10 - Loss: 1.3372
Epoch 7/10 - Loss: 1.2991
Epoch 8/10 - Loss: 1.2664
Epoch 9/10 - Loss: 1.2371
Epoch 10/10 - Loss: 1.2086


[I 2025-06-13 02:46:26,622] Trial 1 finished with value: 0.5691 and parameters: {'lr': 3.863879405158732e-05, 'beta1': 0.8629274032676316, 'beta2': 0.9184397609263895, 'weight_decay': 0.00010881335003635877}. Best is trial 0 with value: 0.7264.


...

Epoch 1/10 - Loss: 1.3971


[I 2025-06-13 03:08:47,923] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.4751


[I 2025-06-13 03:09:02,839] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.7038

 Results for Adam 
Best hyperparameters : {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}
Best accuracy : 72.64%
Execution time : 1656.47 secondes


In [16]:
AMSGrad_study=run_optuna_study("AMSGrad", 20)

[I 2025-06-13 03:09:02,846] A new study created in memory with name: no-name-68987d89-4ff4-4106-96a6-299e970364e3


Epoch 1/10 - Loss: 1.5494
Epoch 2/10 - Loss: 1.2275
Epoch 3/10 - Loss: 1.1024
Epoch 4/10 - Loss: 1.0125
Epoch 5/10 - Loss: 0.9496
Epoch 6/10 - Loss: 0.8919
Epoch 7/10 - Loss: 0.8476
Epoch 8/10 - Loss: 0.7977
Epoch 9/10 - Loss: 0.7589
Epoch 10/10 - Loss: 0.7225


[I 2025-06-13 03:11:34,315] Trial 0 finished with value: 0.7034 and parameters: {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.7034.


Epoch 1/10 - Loss: 1.9325
Epoch 2/10 - Loss: 1.6561
Epoch 3/10 - Loss: 1.5505
Epoch 4/10 - Loss: 1.4784
Epoch 5/10 - Loss: 1.4284
Epoch 6/10 - Loss: 1.3853
Epoch 7/10 - Loss: 1.3521
Epoch 8/10 - Loss: 1.3224
Epoch 9/10 - Loss: 1.2963
Epoch 10/10 - Loss: 1.2714


[I 2025-06-13 03:14:04,616] Trial 1 finished with value: 0.5525 and parameters: {'lr': 3.863879405158732e-05, 'beta1': 0.8629274032676316, 'beta2': 0.9184397609263895, 'weight_decay': 0.00010881335003635877}. Best is trial 0 with value: 0.7034.


...

Epoch 1/10 - Loss: 1.4122


[I 2025-06-13 03:36:25,891] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.4816


[I 2025-06-13 03:36:40,819] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.7153

 Results for AMSGrad 
Best hyperparameters : {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}
Best accuracy : 70.34%
Execution time : 1657.97 secondes


In [17]:
AdamW_study=run_optuna_study("AdamW", 20)

[I 2025-06-13 03:36:40,826] A new study created in memory with name: no-name-6a5e4b02-45d9-4af7-967b-b248d5977840


Epoch 1/10 - Loss: 1.5271
Epoch 2/10 - Loss: 1.1728
Epoch 3/10 - Loss: 1.0220
Epoch 4/10 - Loss: 0.9208
Epoch 5/10 - Loss: 0.8368
Epoch 6/10 - Loss: 0.7640
Epoch 7/10 - Loss: 0.6949
Epoch 8/10 - Loss: 0.6287
Epoch 9/10 - Loss: 0.5720
Epoch 10/10 - Loss: 0.5134


[I 2025-06-13 03:39:11,569] Trial 0 finished with value: 0.7223 and parameters: {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.7223.


Epoch 1/10 - Loss: 1.9214
Epoch 2/10 - Loss: 1.6321
Epoch 3/10 - Loss: 1.5182
Epoch 4/10 - Loss: 1.4407
Epoch 5/10 - Loss: 1.3880
Epoch 6/10 - Loss: 1.3407
Epoch 7/10 - Loss: 1.3031
Epoch 8/10 - Loss: 1.2712
Epoch 9/10 - Loss: 1.2425
Epoch 10/10 - Loss: 1.2148


[I 2025-06-13 03:41:41,453] Trial 1 finished with value: 0.566 and parameters: {'lr': 3.863879405158732e-05, 'beta1': 0.8629274032676316, 'beta2': 0.9184397609263895, 'weight_decay': 0.00010881335003635877}. Best is trial 0 with value: 0.7223.


...

Epoch 1/10 - Loss: 1.3991


[I 2025-06-13 04:03:58,866] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.4762


[I 2025-06-13 04:04:13,672] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.7036

 Results for AdamW 
Best hyperparameters : {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}
Best accuracy : 72.23%
Execution time : 1652.85 secondes


In [18]:
Nadam_study=run_optuna_study("Nadam", 20)

[I 2025-06-13 04:04:13,679] A new study created in memory with name: no-name-cd88e9fe-e3a5-47db-ba62-e09235bea69d


Epoch 1/10 - Loss: 1.5773
Epoch 2/10 - Loss: 1.2148
Epoch 3/10 - Loss: 1.0488
Epoch 4/10 - Loss: 0.9260
Epoch 5/10 - Loss: 0.8301
Epoch 6/10 - Loss: 0.7461
Epoch 7/10 - Loss: 0.6732
Epoch 8/10 - Loss: 0.5991
Epoch 9/10 - Loss: 0.5302
Epoch 10/10 - Loss: 0.4640


[I 2025-06-13 04:06:44,188] Trial 0 finished with value: 0.7232 and parameters: {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.7232.


Epoch 1/10 - Loss: 1.9087
Epoch 2/10 - Loss: 1.6227
Epoch 3/10 - Loss: 1.5090
Epoch 4/10 - Loss: 1.4321
Epoch 5/10 - Loss: 1.3778
Epoch 6/10 - Loss: 1.3316
Epoch 7/10 - Loss: 1.2929
Epoch 8/10 - Loss: 1.2582
Epoch 9/10 - Loss: 1.2296
Epoch 10/10 - Loss: 1.2032


[I 2025-06-13 04:09:15,389] Trial 1 finished with value: 0.5604 and parameters: {'lr': 3.863879405158732e-05, 'beta1': 0.8629274032676316, 'beta2': 0.9184397609263895, 'weight_decay': 0.00010881335003635877}. Best is trial 0 with value: 0.7232.


...

Epoch 1/10 - Loss: 1.4592


[I 2025-06-13 04:31:33,002] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.5424


[I 2025-06-13 04:31:47,892] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.7048

 Results for Nadam 
Best hyperparameters : {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}
Best accuracy : 72.32%
Execution time : 1654.21 secondes


In [19]:
RAdam_study=run_optuna_study("RAdam", 20)

[I 2025-06-13 04:31:47,899] A new study created in memory with name: no-name-4102d839-1453-45ef-9e72-82469c8090e0


Epoch 1/10 - Loss: 1.5793
Epoch 2/10 - Loss: 1.1768
Epoch 3/10 - Loss: 1.0148
Epoch 4/10 - Loss: 0.9024
Epoch 5/10 - Loss: 0.8163
Epoch 6/10 - Loss: 0.7424
Epoch 7/10 - Loss: 0.6664
Epoch 8/10 - Loss: 0.5972
Epoch 9/10 - Loss: 0.5310
Epoch 10/10 - Loss: 0.4736


[I 2025-06-13 04:34:18,645] Trial 0 finished with value: 0.7313 and parameters: {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}. Best is trial 0 with value: 0.7313.


Epoch 1/10 - Loss: 1.9601
Epoch 2/10 - Loss: 1.6374
Epoch 3/10 - Loss: 1.5181
Epoch 4/10 - Loss: 1.4380
Epoch 5/10 - Loss: 1.3834
Epoch 6/10 - Loss: 1.3346
Epoch 7/10 - Loss: 1.2962
Epoch 8/10 - Loss: 1.2639
Epoch 9/10 - Loss: 1.2347
Epoch 10/10 - Loss: 1.2061


[I 2025-06-13 04:36:48,636] Trial 1 finished with value: 0.5686 and parameters: {'lr': 3.863879405158732e-05, 'beta1': 0.8629274032676316, 'beta2': 0.9184397609263895, 'weight_decay': 0.00010881335003635877}. Best is trial 0 with value: 0.7313.


...

Epoch 1/10 - Loss: 1.4138


[I 2025-06-13 04:56:51,018] Trial 18 pruned. 


Epoch 1/10 - Loss: 1.5608


[I 2025-06-13 04:57:05,928] Trial 19 pruned. 


Epoch 1/10 - Loss: 1.7639

 Results for RAdam 
Best hyperparameters : {'lr': 0.00046568046379195655, 'beta1': 0.9508454290819022, 'beta2': 0.9000113231069171, 'weight_decay': 8.072307884499753e-05}
Best accuracy : 73.13%
Execution time : 1518.03 secondes


### Visualisation and saves

In [111]:
def visualization (study, params) :
    vis.plot_optimization_history(study).show()
    vis.plot_param_importances(study).show()
    vis.plot_parallel_coordinate(study).show()
    vis.plot_intermediate_values(study).show()
    contour_params = list(itertools.combinations(params, 2))        
    for p1, p2 in contour_params:
            vis.plot_contour(study, params=[p1, p2])

In [112]:
def save_optuna_plots(study, params, save_dir="optuna_plots"):
    os.makedirs(save_dir, exist_ok=True)
    
    def save_fig(fig, name):
        path = os.path.join(save_dir, f"{name}.html")
        fig.write_html(path)
        print(f"Saved {name} plot to {path}")
        
    save_fig(vis.plot_optimization_history(study), "optimization_history")
    save_fig(vis.plot_param_importances(study), "param_importances")
    save_fig(vis.plot_parallel_coordinate(study), "parallel_coordinate")
    save_fig(vis.plot_intermediate_values(study), "intermediate_values")
    contour_params = list(itertools.combinations(params, 2))        
    for p1, p2 in contour_params:
        save_fig(vis.plot_contour(study, params=[p1, p2]), f"contour_{p1}_{p2}")

In [114]:
visualization(SGD_study, ["lr", "weight_decay"])
save_optuna_plots(SGD_study, ["lr", "weight_decay"], "figs_optuna/SGD")

Saved optimization_history plot to figs_optuna/SGD/optimization_history.html
Saved param_importances plot to figs_optuna/SGD/param_importances.html
Saved parallel_coordinate plot to figs_optuna/SGD/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/SGD/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/SGD/contour_lr_weight_decay.html


In [115]:
visualization(AdaGrad_study, ["lr", "weight_decay"])
save_optuna_plots(SGD_study, ["lr", "weight_decay"], "figs_optuna/AdaGrad")

Saved optimization_history plot to figs_optuna/AdaGrad/optimization_history.html
Saved param_importances plot to figs_optuna/AdaGrad/param_importances.html
Saved parallel_coordinate plot to figs_optuna/AdaGrad/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/AdaGrad/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/AdaGrad/contour_lr_weight_decay.html


In [116]:
visualization(RMSprop_study, ["lr", "weight_decay", "alpha"])
save_optuna_plots(RMSprop_study, ["lr", "weight_decay", "alpha"], "figs_optuna/RMSprop")

Saved optimization_history plot to figs_optuna/RMSprop/optimization_history.html
Saved param_importances plot to figs_optuna/RMSprop/param_importances.html
Saved parallel_coordinate plot to figs_optuna/RMSprop/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/RMSprop/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/RMSprop/contour_lr_weight_decay.html
Saved contour_lr_alpha plot to figs_optuna/RMSprop/contour_lr_alpha.html
Saved contour_weight_decay_alpha plot to figs_optuna/RMSprop/contour_weight_decay_alpha.html


In [171]:
visualization(Adam_study, ["lr", "weight_decay", "beta1", "beta2"])
save_optuna_plots(Adam_study, ["lr", "weight_decay", "beta1", "beta2"], "figs_optuna/Adam")

Saved optimization_history plot to figs_optuna/Adam_vrai/optimization_history.html
Saved param_importances plot to figs_optuna/Adam_vrai/param_importances.html
Saved parallel_coordinate plot to figs_optuna/Adam_vrai/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/Adam_vrai/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/Adam_vrai/contour_lr_weight_decay.html
Saved contour_lr_beta1 plot to figs_optuna/Adam_vrai/contour_lr_beta1.html
Saved contour_lr_beta2 plot to figs_optuna/Adam_vrai/contour_lr_beta2.html
Saved contour_weight_decay_beta1 plot to figs_optuna/Adam_vrai/contour_weight_decay_beta1.html
Saved contour_weight_decay_beta2 plot to figs_optuna/Adam_vrai/contour_weight_decay_beta2.html
Saved contour_beta1_beta2 plot to figs_optuna/Adam_vrai/contour_beta1_beta2.html


In [170]:
visualization(AMSGrad_study, ["lr", "weight_decay", "beta1", "beta2"])
save_optuna_plots(AMSGrad_study, ["lr", "weight_decay", "beta1", "beta2"], "figs_optuna/AMSGrad")

Saved optimization_history plot to figs_optuna/AMSGrad/optimization_history.html
Saved param_importances plot to figs_optuna/AMSGrad/param_importances.html
Saved parallel_coordinate plot to figs_optuna/AMSGrad/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/AMSGrad/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/AMSGrad/contour_lr_weight_decay.html
Saved contour_lr_beta1 plot to figs_optuna/AMSGrad/contour_lr_beta1.html
Saved contour_lr_beta2 plot to figs_optuna/AMSGrad/contour_lr_beta2.html
Saved contour_weight_decay_beta1 plot to figs_optuna/AMSGrad/contour_weight_decay_beta1.html
Saved contour_weight_decay_beta2 plot to figs_optuna/AMSGrad/contour_weight_decay_beta2.html
Saved contour_beta1_beta2 plot to figs_optuna/AMSGrad/contour_beta1_beta2.html


In [125]:
visualization(AdamW_study, ["lr", "weight_decay", "beta1", "beta2"])
save_optuna_plots(AdamW_study, ["lr", "weight_decay", "beta1", "beta2"], "figs_optuna/AdamW")

Saved optimization_history plot to figs_optuna/AdamW/optimization_history.html
Saved param_importances plot to figs_optuna/AdamW/param_importances.html
Saved parallel_coordinate plot to figs_optuna/AdamW/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/AdamW/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/AdamW/contour_lr_weight_decay.html
Saved contour_lr_beta1 plot to figs_optuna/AdamW/contour_lr_beta1.html
Saved contour_lr_beta2 plot to figs_optuna/AdamW/contour_lr_beta2.html
Saved contour_weight_decay_beta1 plot to figs_optuna/AdamW/contour_weight_decay_beta1.html
Saved contour_weight_decay_beta2 plot to figs_optuna/AdamW/contour_weight_decay_beta2.html
Saved contour_beta1_beta2 plot to figs_optuna/AdamW/contour_beta1_beta2.html


In [126]:
visualization(Nadam_study, ["lr", "weight_decay", "beta1", "beta2"])
save_optuna_plots(Nadam_study, ["lr", "weight_decay", "beta1", "beta2"], "figs_optuna/Nadam")

Saved optimization_history plot to figs_optuna/Nadam/optimization_history.html
Saved param_importances plot to figs_optuna/Nadam/param_importances.html
Saved parallel_coordinate plot to figs_optuna/Nadam/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/Nadam/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/Nadam/contour_lr_weight_decay.html
Saved contour_lr_beta1 plot to figs_optuna/Nadam/contour_lr_beta1.html
Saved contour_lr_beta2 plot to figs_optuna/Nadam/contour_lr_beta2.html
Saved contour_weight_decay_beta1 plot to figs_optuna/Nadam/contour_weight_decay_beta1.html
Saved contour_weight_decay_beta2 plot to figs_optuna/Nadam/contour_weight_decay_beta2.html
Saved contour_beta1_beta2 plot to figs_optuna/Nadam/contour_beta1_beta2.html


In [143]:
visualization(RAdam_study, ["lr", "weight_decay", "beta1", "beta2"])
save_optuna_plots(RAdam_study, ["lr", "weight_decay", "beta1", "beta2"], "figs_optuna/RAdam")

Saved optimization_history plot to figs_optuna/RAdam/optimization_history.html
Saved param_importances plot to figs_optuna/RAdam/param_importances.html
Saved parallel_coordinate plot to figs_optuna/RAdam/parallel_coordinate.html
Saved intermediate_values plot to figs_optuna/RAdam/intermediate_values.html
Saved contour_lr_weight_decay plot to figs_optuna/RAdam/contour_lr_weight_decay.html
Saved contour_lr_beta1 plot to figs_optuna/RAdam/contour_lr_beta1.html
Saved contour_lr_beta2 plot to figs_optuna/RAdam/contour_lr_beta2.html
Saved contour_weight_decay_beta1 plot to figs_optuna/RAdam/contour_weight_decay_beta1.html
Saved contour_weight_decay_beta2 plot to figs_optuna/RAdam/contour_weight_decay_beta2.html
Saved contour_beta1_beta2 plot to figs_optuna/RAdam/contour_beta1_beta2.html
